In [ ]:
#!pip -q install fitz
!pip -q install transformers
!pip -q install torch
!pip install -q tiktoken einops accelerate bitsandbytes
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0

In [ ]:
import tempfile
import fitz
import json
import torch
import transformers
from transformers import pipeline,BitsAndBytesConfig
import re

In [ ]:
# ✅ STEP 1: Install requirements
!pip install llama-cpp-python==0.2.24 PyMuPDF tqdm --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 47.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl size=5959970 sha256=6d7b3a44b6c4fa8141bb0dbb266cacf29b726fdc98364f31eee6ce90d8227bdb
  Stored in directory: /root/.cache/pip/wheels/c0/03/66/eb3810eafd55d921b2be32896d1f44313996982360663aa80b
Successfully built llama-cpp-python
  Attempting uninstall: llama-cpp-python
    Found existing installation: llama_cpp_python 0.2.24
    Uninstalling llama_cpp_python-0.2.24:
      Successfully uninstalled llama_cpp_python-0.2.24


In [ ]:
from huggingface_hub import login

# Paste your token when prompted
login()


In [ ]:
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(
    repo_id="bartowski/Phi-3.5-mini-instruct-GGUF",
    filename="Phi-3.5-mini-instruct-Q5_K_S.gguf",
    local_dir="/content/models/phi3.5",
    local_dir_use_symlinks=False
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:933: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Phi-3.5-mini-instruct-Q5_K_S.gguf:   0%|          | 0.00/2.64G [00:00<?, ?B/s]

In [ ]:
from llama_cpp import Llama

def load_phi_model(model_path):
    try:
        llm = Llama(
            model_path=model_path,  # Path to your GGUF model
            n_ctx=4096,             # Context window size
            n_threads=8,            # Number of threads
            n_gpu_layers=0,         # Set to 0 if using CPU-only
            use_mmap=False,         # Use memory-mapped file (disabled here)
            verbose=True            # Set to True for verbose output
        )
        return llm
    except Exception as e:
        print("Error loading model:", e)
        return None

model_path = "/content/models/phi3.5/Phi-3.5-mini-instruct-Q5_K_S.gguf"
model = load_phi_model(model_path)

if model is None:
    print("Model loading failed.")
else:
    print("Model loaded successfully.")


llama_model_loader: loaded meta data with 40 key-value pairs and 197 tensors from /content/models/phi3.5/Phi-3.5-mini-instruct-Q5_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Phi 3.5 Mini Instruct
llama_model_loader: - kv   3:                           general.finetune str              = instruct
llama_model_loader: - kv   4:                           general.basename str              = Phi-3.5
llama_model_loader: - kv   5:                         general.size_label str              = mini
llama_model_loader: - kv   6:                            general.license str              = mit
llama_model_loader: - kv   7:    

Model loaded successfully.


CPU : SSE3 = 1 | SSSE3 = 1 | AVX = 1 | AVX2 = 1 | F16C = 1 | FMA = 1 | BMI2 = 1 | LLAMAFILE = 1 | OPENMP = 1 | AARCH64_REPACK = 1 | 
Model metadata: {'quantize.imatrix.chunks_count': '151', 'quantize.imatrix.entries_count': '128', 'quantize.imatrix.dataset': '/training_dir/calibration_datav3.txt', 'quantize.imatrix.file': '/models_out/Phi-3.5-mini-instruct-GGUF/Phi-3.5-mini-instruct.imatrix', 'general.quantization_version': '2', 'tokenizer.chat_template': "{% for message in messages %}{% if message['role'] == 'system' and message['content'] %}{{'<|system|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'user' %}{{'<|user|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>\n' + message['content'] + '<|end|>\n'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% else %}{{ eos_token }}{% endif %}", 'phi3.rope.scaling.original_context_length': '4096', 'general.architecture': 'phi3', 'phi3.rope.sca

In [ ]:
def load_phi_model(model_path):
    llm = Llama(
    model_path="/content/models/phi3.5/Phi-3.5-mini-instruct-Q5_K_S.gguf",
    n_ctx=4096,
    n_threads=8,
    n_gpu_layers=0,  # Reduce to 0 if CPU-only
    use_mmap=True,
    verbose=False
)
    return llm


In [ ]:
import json
import re
import fitz  # PyMuPDF
from llama_cpp import Llama  # Make sure to install llama-cpp

# Extract clean JSON from output
def extract_json(text):
    try:
        return json.loads(text)
    except Exception as e:
        return {"Error": f"Failed to extract JSON: {str(e)}"}

# Build prompt suited for TinyLLaMA-style chat model
def build_prompt(text):
    instruction = """
You are an information extraction engine. Return ONLY valid JSON, no explanations.

JSON Structure:
{
  "Title": "Paper title",
  "Authors": ["Author 1", "Author 2"],
  "DOI": "DOI if available",
  "Keywords": ["Keyword1", "Keyword2"],
  "Abstract": "Abstract text",
  "Document Type": "Research Paper, Thesis, etc.",
  "Number of References": 10
}

Extract metadata from the following scientific paper:
"""
    return f"<|user|>\n{instruction.strip()}\n{text[:2000]}\n<|assistant|>"


# Call model and extract structured data
def extract_metadata(generator, paper_text):
    prompt = build_prompt(paper_text)
    response = generator(
        prompt,
        max_tokens=1024,             # allow enough room for full JSON
        temperature=0,               # deterministic output
        top_p=1.0,
        stop=["<|end|>", "</s>"],    # prevent model from rambling
    )


    # print(response)
    raw_output = response["choices"][0]["text"]

    print("\n==== RAW MODEL OUTPUT ====")
    print(raw_output)
    print("==== END RAW OUTPUT ====\n")

    return extract_json(raw_output)

# Extract raw text from PDF using PyMuPDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text("text") for page in doc)
    return text if text.strip() else "Error: No extractable text found in PDF."

# Load LLaMA 3 GGUF Model

# Path to your GGUF model
model_path = "/content/models/phi3.5/Phi-3.5-mini-instruct.Q8_0.gguf"
# Load model
model = load_phi_model(model_path)

if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/1.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


llama_init_from_model: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized



==== RAW MODEL OUTPUT ====
 {
  "Title": "Extracting Scientific Figures with Distantly Supervised Neural Networks",
  "Authors": ["Noah Siegel", "Nicholas Lourie", "Russell Power", "Waleed Ammar"],
  "DOI": "Not provided",
  "Keywords": ["Figure Extraction", "Distant Supervision", "Deep Learning", "Neural Networks", "Computer Vision"],
  "Abstract": "Non-textual components such as charts, diagrams and tables provide key information in many scientific documents. However, the lack of large labeled datasets has impeded the development of data-driven methods for scientific figure extraction. In this paper, we induce high-quality training labels for the task of figure extraction in a large number of scientific documents, with no human intervention. We leverage the auxiliary data provided in two large web collections of scientific documents (arXiv and PubMed) to locate figures and their associated captions in the rasterized PDF. We share the resulting dataset of over 5.5 million induced lab

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/2.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Automatic Recognition of Learning Resource",
  "Authors": ["Soumya Banerjee", "Debarshi Kumar Sanyal", "Samiran Chattopadhyay", "Plaban Kumar Bhowmick", "Partha Pratim Das"],
  "DOI": null,
  "Keywords": ["deep learning", "transfer learning", "digital library"],
  "Abstract": "Digital libraries generally need to process a large volume of diverse document types. The collection and tagging of metadata is a long, error-prone, manpower-consuming task. We are attempting to build an automatic metadata extractor for digital libraries. In this work, we present the Heterogeneous Learning Resources (HLR) dataset for document image classification. The individual learning resource is first decomposed into its constituent document images (sheets) which are then passed through an OCR tool to obtain the textual representation. The document image and its textual content are classified with state-of-the-art classifiers. Finally, the labels of the constituent d

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/3.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Model Selection with Model Zoo via Graph Learning",
  "Authors": ["Ziyu Li", "Hilco van der Wilk", "Danning Zhan", "Megha Khosla", "Alessandro Bozzon", "Rihan Hai"],
  "DOI": "https://doi.org/10.1016/j.mlr.2023.01.001",
  "Keywords": ["Deep Learning", "Model Selection", "Model Zoo", "Graph Learning", "TransferGraph"],
  "Abstract": "Pre-trained deep learning (DL) models are increasingly accessible in public repositories, i.e., model zoos. Given a new prediction task, finding the best model to fine-tune can be computationally intensive and costly, especially when the number of pre-trained models is large. Selecting the right pre-trained models is crucial, yet complicated by the diversity of models from various model families (like ResNet, Vit, Swin) and the hidden relationships between models and datasets. Existing methods, which utilize basic information from models and datasets to compute scores indicating model performance on target datasets

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/4.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Extracting Decision Model and Notation models from text using deep learning techniques",
  "Authors": ["Alexandre Goossens", "Johannes De Smedt", "Jan Vanthienen"],
  "DOI": "https://doi.org/10.1016/j.eswa.2022.118667",
  "Keywords": ["Deep learning", "Decision Model and Notation", "DMN", "Decision model extraction"],
  "Abstract": "Companies and organizations often use manuals and guidelines to communicate and execute operational decisions. Decision Model and Notation (DMN) models can be used to model and automate these decisions. Modeling a decision from a textual source, however, is a time intensive and complex activity hence a need for shorter modeling times. This paper studies how NLP deep learning techniques can extract decision models from text faster. In this paper, we study and evaluate an automatic sentence classifier and a decision dependency extractor using NLP deep learning models (BERT and Bi-LSTM-CRF). A large labeled and tagged

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/5.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Automated data extraction with conversational LLMs for materials science",
  "Authors": ["Maciej P. Polak", "Dane Morgan"],
  "DOI": "10.1038/s41467-024-45914-8",
  "Keywords": ["Automated data extraction", "Conversational LLMs", "Materials science", "Data accuracy", "ChatExtract"],
  "Abstract": "This work introduces ChatExtract, a method for automating accurate data extraction from research papers using conversational language models. ChatExtract employs a set of engineered prompts to identify, extract, and verify data through follow-up questions, addressing issues of factual inaccuracy. The method demonstrates high precision and recall, achieving near-90% accuracy with LLMs like GPT-4. The paper also presents databases for critical cooling rates of metallic glasses and yield strengths of high entropy alloys developed using ChatExtract. The simplicity, transferability, and accuracy of ChatExtract suggest its potential as a powerful tool for 

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/6.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Smart Learning Environments: Prerequisites-based course recommendation using concept prerequisites and metadata matching",
  "Authors": ["Abdessamad Chanaa", "Nour-eddine El Faddouli"],
  "DOI": "10.1186/s40561-024-00301-0",
  "Keywords": ["Smart Learning Environments", "Prerequisites-based course recommendation", "Concept prerequisites", "Metadata matching"],
  "Abstract": "The recommendation is an active area of scientific research; it is also a challenging and fundamental problem in online education. However, classical recommender systems usually suffer from item cold-start issues. Besides, unlike other fields like e-commerce or entertainment, e-learning recommendations must ensure that learners have the adequate background knowledge to cognitively receive the recommended learning objects. For that reason, when designing an efficient e-learning recommendation method, these challenges should be considered. To address those issues, in this pa

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/7.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Deep Transfer Learning for Metadata Extraction from German Publications",
  "Authors": ["Zeyd Boukhers", "Nada Beili", "Timo Hartmann", "Prantik Goswami", "Muhammad Arslan Zafar"],
  "DOI": null,
  "Keywords": ["transfer learning", "metadata extraction", "neural networks"],
  "Abstract": "In contrast to most of the English scientiﬁc publications that follow standard and simple layouts, the order, content, position and size of metadata in German publications vary greatly among publications. This variety makes traditional NLP methods fail to accurately extract metadata from these publications. In this paper, we present a method that extracts metadata from PDF documents with different layouts and styles by viewing the document as an image. We used Mask R-CNN which is trained on COCO dataset and finetuned with PubLayNet dataset that consists of 200K PDF snapshots with five basic classes (e.g. text, figure, etc). We refine-tuned the model on our pr

In [ ]:
import json
import re
import fitz  # PyMuPDF
from llama_cpp import Llama  # Make sure to install llama-cpp

# Extract clean JSON from output
def extract_json(text):
    text = re.sub(r'(?<!\\)\n', ' ', text)
    text = text.encode('utf-8', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r"\(.,*?\)", "", text)  # Remove any text inside parentheses
    text = re.sub(r"[\n\t]", " ", text)  # Remove newline and tab characters
    text = re.sub(r"(ERNIE 3\.0):", r"\1", text)
    print("raw text here",text)
    try:
        return json.loads(text)
    except Exception as e:
        return {"Error": f"Failed to extract JSON: {str(e)}"}

# Build prompt suited for TinyLLaMA-style chat model
def build_prompt(text):
    instruction = """
You are an information extraction engine. Return ONLY valid JSON, no explanations.

JSON Structure:
{
  "Title": "Paper title",
  "Authors": ["Author 1", "Author 2"],
  "DOI": "DOI if available",
  "Keywords": ["Keyword1", "Keyword2"],
  "Abstract": "Abstract text",
  "Document Type": "Research Paper, Thesis, etc.",
  "Number of References": 10
}

Extract metadata from the following scientific paper:
"""
    return f"<|user|>\n{instruction.strip()}\n{text[:2000]}\n<|assistant|>"


# Call model and extract structured data
def extract_metadata(generator, paper_text):
    prompt = build_prompt(paper_text)
    response = generator(
        prompt,
        max_tokens=1024,             # allow enough room for full JSON
        temperature=0,               # deterministic output
        top_p=1.0,
        stop=["<|end|>", "</s>"],    # prevent model from rambling
    )


    # print(response)
    raw_output = response["choices"][0]["text"]

    print("\n==== RAW MODEL OUTPUT ====")
    print(raw_output)
    print("==== END RAW OUTPUT ====\n")

    return extract_json(raw_output)

# Extract raw text from PDF using PyMuPDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text("text") for page in doc)
    return text if text.strip() else "Error: No extractable text found in PDF."

# Load LLaMA 3 GGUF Model

# Path to your GGUF model
model_path = "/content/models/phi3.5/Phi-3.5-mini-instruct.Q8_0.gguf"
# Load model
model = load_phi_model(model_path)

if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/8.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


llama_init_from_model: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized



==== RAW MODEL OUTPUT ====
 {
  "Title": "ERNIE 3.0: LARGE-SCALE KNOWLEDGE ENHANCED
PRE-TRAINING FOR LANGUAGE UNDERSTANDING AND
GENERATION",
  "Authors": ["Yu Sun", "Shuohuan Wang", "Shikun Feng", "Siyu Ding", "Chao Pang", "Junyuan Shang", "Jiaxiang Liu", "Xuyi Chen", "Yanbin Zhao", "Yuxiang Lu", "Weixin Liu", "Zhihua Wu", "Weibao Gong", "Jianzhong Liang", "Zhizhou Shang", "Peng Sun", "Wei Liu", "Xuan Ouyang", "Dianhai Yu", "Hao Tian", "Hua Wu", "Haifeng Wang"],
  "DOI": "Not provided",
  "Keywords": ["ERNIE 3.0", "large-scale knowledge enhanced models", "pre-training", "language understanding", "language generation", "zero-shot learning", "few-shot learning", "auto-regressive network", "auto-encoding network"],
  "Abstract": "Pre-trained models have achieved state-of-the-art results in various Natural Language Processing
(NLP) tasks. Recent works such as T5 [1] and GPT-3 [2] have shown that scaling up pre-trained
language models can improve their generalization abilities. Particularl

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/9.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Few-shot named entity recognition framework for forestry science metadata extraction",
  "Authors": ["Yuquan Fan", "Hong Xiao", "Min Wang", "Junchi Wang", "Wenchao Jiang", "Chang Zhu"],
  "DOI": "10.1007/s12652-023-04740-4",
  "Keywords": ["Few-shot Named Entity Recognition", "Metadata Extraction", "Forestry Science", "Data Augmentation", "Multi-granularity Dilated Convolution Neural Network"],
  "Abstract": "The effective utilization of accumulated forestry science papers is of paramount significance in enhancing our understanding of the current state of forests and the formulation of strategies for forest environmental preservation. However, the present challenge lies in the deficient richness of metadata associated with these pivotal documents, rendering their comprehensive exploitation a formidable endeavor. Metadata from forestry science papers serves as a foundational cornerstone for the efficient management and utilization of these scho

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/10.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Extracting structured knowledge from scientific text with large language models",
  "Authors": ["John Dagdelen", "Alexander Dunn", "Sanghoon Lee", "Nicholas Walker", "Andrew S. Rosen", "Gerbrand Ceder", "Kristin A. Persson", "Anubhav Jain"],
  "DOI": "10.1038/s41467-024-45563-x",
  "Keywords": ["structured knowledge extraction", "named entity recognition", "relation extraction", "large language models", "GPT-3", "Llama-2", "materials chemistry", "dopants", "host materials", "metal-organic frameworks", "composition", "phase", "morphology", "application"],
  "Abstract": "Extracting structured knowledge from scientific text remains a challenging task for machine learning models. Here, we present a simple approach to joint named entity recognition and relation extraction and demonstrate how pre-trained large language models (GPT-3, Llama-2) can be fine-tuned to extract useful records of complex scientific knowledge. We test three representative ta

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/11.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Extracting structured knowledge from scientific text with large language models",
  "Authors": ["John Dagdelen", "Alexander Dunn", "Sanghoon Lee", "Nicholas Walker", "Andrew S. Rosen", "Gerbrand Ceder", "Kristin A. Persson", "Anubhav Jain"],
  "DOI": "10.1038/s41467-024-45563-x",
  "Keywords": ["structured knowledge extraction", "named entity recognition", "relation extraction", "large language models", "GPT-3", "Llama-2", "materials chemistry", "dopants", "host materials", "metal-organic frameworks", "composition", "phase", "morphology", "application"],
  "Abstract": "Extracting structured knowledge from scientific text remains a challenging task for machine learning models. Here, we present a simple approach to joint named entity recognition and relation extraction and demonstrate how pre-trained large language models (GPT-3, Llama-2) can be fine-tuned to extract useful records of complex scientific knowledge. We test three representative ta

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/12.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Information Retrieval Meets Large Language Models: A Strategic Report from Chinese IR Community",
  "Authors": [
    "Qingyao AIa",
    "Ting BAIb",
    "Zhao CAOc",
    "Yi CHANGd",
    "Jiawei CHEN(́)",
    "Zhumin CHENf",
    "Zhiyong CHENGg",
    "Shoubin DONGh",
    "Zhicheng DOUi",
    "Fuli FENG j",
    "Shen GAO f",
    "Jiafeng GUOk",
    "Xiangnan HE(́)",
    "Yanyan LANa",
    "Chenliang LIl",
    "Yiqun LIUa",
    "Ziyu LYUm",
    "Weizhi MAa",
    "Jun MAf",
    "Zhaochun REN f",
    "Pengjie REN f",
    "Zhiqiang WANGn",
    "Mingwen WANGo",
    "Ji-Rong WENi",
    "Le WUp",
    "Xin XIN f",
    "Jun XUi",
    "Dawei YINq",
    "Peng ZHANG(́)",
    "Fan ZHANGl",
    "Weinan ZHANGs",
    "Min ZHANGa",
    "Xiaofei ZHUt",
    "Tsinghua University",
    "Beijing University of Posts and Telecommunications",
    "Huawei Technologies Ltd. Co",
    "Jilin University",
    "Zhejiang University",
    "Shandong University",
    "Shandong A

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/13.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Building an annotated corpus for automatic metadata extraction from multilingual journal article references",
  "Authors": ["Wonjun Choi", "Hwa-Mook Yoon", "Mi-Hwan Hyun", "Hye-Jin Lee", "Jae-Wook Seol", "Kangsan Dajeong Lee", "Young Joon Yoon", "Hyesoo Kong"],
  "DOI": null,
  "Keywords": ["metadata extraction", "multilingual references", "automatic annotation", "BERT-based model", "GROBID"],
  "Abstract": "Bibliographic references containing citation information of academic literature play an important role as a medium connecting earlier and recent studies. As references contain machine-readable metadata such as author name, title, or publication year, they have been widely used in the field of citation information services including search services for scholarly information and research trend analysis. Many institutions around the world manually extract and continuously accumulate reference metadata to provide various scholarly services. Ho

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/14.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "LAME: Layout-Aware Metadata Extraction Approach for Research Articles",
  "Authors": ["Jongyun Cho", "Hyesoo Kong", "Hwamook Yoon", "Heung-seon Oh", "Yuchul Jung"],
  "DOI": "Not provided",
  "Keywords": ["Automatic layout analysis", "Layout-MetaBERT", "Metadata extraction", "Research article"],
  "Abstract": "The volume of academic literature, such as academic conference papers and journals, has increased rapidly worldwide, and research on metadata extraction is ongoing. However, high-performing metadata extraction is still challenging due to diverse layout formats according to journal publishers. To accommodate the diversity of the layouts of academic journals, we propose a novel LAyout-aware Metadata Extraction (LAME) framework equipped with the three characteristics (e.g., design of an automatic layout analysis, construction of a large meta-data training set, and construction of Layout-MetaBERT). We designed an automatic layout analysis us

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/15.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Layout Aware Semantic Element Extraction for Sustainable Science & Technology Decision Support",
  "Authors": ["Hyuntae Kim", "Jongyun Choi", "Soyoung Park", "Yuchul Jung"],
  "DOI": "10.3390/su14052802",
  "Keywords": ["Science & Technology", "Automated Text Mining", "Vision Recognition", "Knowledge Graph Construction"],
  "Abstract": "New scientiﬁc and technological (S&T) knowledge is being introduced rapidly, and hence, analysis efforts to understand and analyze new published S&T documents are increasing daily. Automated text mining and vision recognition techniques alleviate the burden somewhat, but the various document layout formats and knowledge content granularities across the S&T ﬁeld make it challenging. Therefore, this paper proposes LA-SEE (LAME and Vi-SEE), a knowledge graph construction framework that simultaneously extracts meta-information and useful image objects from S&T documents in various layout formats. We adopt Layout-aw

In [ ]:
import json
import re
import fitz  # PyMuPDF
from llama_cpp import Llama  # Make sure to install llama-cpp

# Extract clean JSON from output
def extract_json(text):
    cleaned_output = re.sub(r'(?<!\\)\n', ' ', text)
    try:
        return json.loads(text)
    except Exception as e:
        return {"Error": f"Failed to extract JSON: {str(e)}"}

# Build prompt suited for TinyLLaMA-style chat model
def build_prompt(text):
    instruction = """
You are an information extraction engine. Return ONLY valid JSON, no explanations.

JSON Structure:
{
  "Title": "Paper title",
  "Authors": ["Author 1", "Author 2"],
  "DOI": "DOI if available",
  "Keywords": ["Keyword1", "Keyword2"],
  "Abstract": "Abstract text",
  "Document Type": "Research Paper, Thesis, etc.",
  "Number of References": 10
}

Extract metadata from the following scientific paper:
"""
    return f"<|user|>\n{instruction.strip()}\n{text[:2000]}\n<|assistant|>"


# Call model and extract structured data
def extract_metadata(generator, paper_text):
    prompt = build_prompt(paper_text)
    response = generator(
        prompt,
        max_tokens=1024,             # allow enough room for full JSON
        temperature=0,               # deterministic output
        top_p=1.0,
        stop=["<|end|>", "</s>"],    # prevent model from rambling
    )


    # print(response)
    raw_output = response["choices"][0]["text"]

    print("\n==== RAW MODEL OUTPUT ====")
    print(raw_output)
    print("==== END RAW OUTPUT ====\n")

    return extract_json(raw_output)

# Extract raw text from PDF using PyMuPDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text("text") for page in doc)
    return text if text.strip() else "Error: No extractable text found in PDF."

# Load LLaMA 3 GGUF Model

# Path to your GGUF model
model_path = "/content/models/phi3.5/Phi-3.5-mini-instruct.Q8_0.gguf"
# Load model
model = load_phi_model(model_path)


if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/16.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


llama_init_from_model: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized



==== RAW MODEL OUTPUT ====
 {
  "Title": "Convolutional Neural Networks for Sentence Classification",
  "Authors": ["Yoon Kim"],
  "DOI": null,
  "Keywords": ["Convolutional Neural Networks", "Sentence Classification", "Word Vectors", "Fine-tuning"],
  "Abstract": "We report on a series of experiments with convolutional neural networks (CNN) trained on top of pre-trained word vectors for sentence-level classification tasks. We show that a simple CNN with little hyperparameter tuning and static vectors achieves excellent results on multiple benchmarks. Learning task-specific vectors through fine-tuning offers further gains in performance. We additionally propose a simple modification to the architecture to allow for the use of both task-specific and static vectors. The CNN models discussed herein improve upon the state of the art on 4 out of 7 tasks, which include sentiment analysis and question classification.",
  "Document Type": "Research Paper",
  "Number of References": null
}





In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/17.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Employing Conditional Random Fields for Information Extraction in Research Papers",
  "Authors": ["Fuchun Peng", "Andrew McCallum"],
  "DOI": "Not provided",
  "Keywords": ["Information extraction", "Constraint information extraction", "Conditional random fields", "Regularization"],
  "Abstract": "With the increasing use of research paper search engines, such as CiteSeer, for both literature search and hiring decisions, the accuracy of such systems is of paramount importance. This article employs conditional random fields (CRFs) for the task of extracting various common fields from the headers and citations of research papers. CRFs provide a principled way for incorporating various local features, external lexicon features, and global layout features. The basic theory of CRFs is becoming well-understood, but best-practices for applying them to real-world data requires additional exploration. We make an empirical exploration of several factors,

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/18.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Information extraction from scientific articles: a survey",
  "Authors": ["Zara Nasar", "Syed Waqar Jaffry", "Muhammad Kamran Malik"],
  "DOI": "https://doi.org/10.1007/978-3-030-20654-5_1",
  "Keywords": ["Information extraction", "Scientific articles", "World Wide Web", "Automatic analysis", "Metadata", "Key-insights", "Benchmark datasets", "Computational approaches", "Rule-based approaches", "Hidden Markov Models", "Conditional Random Fields", "Support Vector Machines", "Naive-Bayes classification", "Deep Learning"],
  "Abstract": "In last few decades, with the advent of World Wide Web (WWW), world is being over-loaded with huge data. This huge data carries potential information that once extracted, can be used for betterment of humanity. Information from this data can be extracted using manual and automatic analysis. Manual analysis is not scalable and efﬁcient, whereas, the automatic analysis involves computing mechanisms that aid in auto

In [ ]:
import json
import re
import fitz  # PyMuPDF
from llama_cpp import Llama  # Make sure to install llama-cpp

# Extract clean JSON from output
def extract_json(text):
    match = re.search(r'\{.*\}', text, re.DOTALL)
    if match:
        json_str = match.group()
        try:
            metadata = json.loads(json_str)
            return metadata

        except Exception as e:
            return {"Error": f"Failed to extract JSON: {str(e)}"}
    else:
        return {"Error": "No valid JSON object found in the text."}



    # text = text.strip()
    # text = re.sub(r'(?<!\\)\n', ' ', text)
    # text = re.sub(r'(?i)^json\s*', '', text.strip())             # Remove "JSON:" or "json"
    # text = re.sub(r'```json', '', text, flags=re.IGNORECASE)     # Remove ```json
    # text = re.sub(r'```', '', text)
    # text = text.encode('utf-8', 'ignore').decode('utf-8', 'ignore')
    # try:
    #     return json.loads(text)
    # except Exception as e:
    #     return {"Error": f"Failed to extract JSON: {str(e)}"}

# Build prompt suited for TinyLLaMA-style chat model
# def build_prompt(text):
#     instruction = """
# You are an information extraction engine. Return ONLY valid JSON, no explanations.

# JSON Structure:
# {
#   "Title": "Paper title",
#   "Authors": ["Author 1", "Author 2"],
#   "DOI": "DOI if available",
#   "Keywords": ["Keyword1", "Keyword2"],
#   "Abstract": "Abstract text",
#   "Document Type": "Research Paper, Thesis, etc.",
#   "Number of References": 10
# }

# Extract metadata from the following scientific paper:
# """
#     return f"<|user|>\n{instruction.strip()}\n{text[:2000]}\n<|assistant|>"

def build_prompt(text):
    return f"""Extract the following metadata from the paper below and output ONLY valid JSON:

Title
Authors
DOI
Keywords
Abstract
Document Type
Number of References

Paper content:
{text[:2000]}
"""


# Call model and extract structured data
def extract_metadata(generator, paper_text):
    prompt = build_prompt(paper_text)
    response = generator(
        prompt,
        max_tokens=1024,             # allow enough room for full JSON
        temperature=0,               # deterministic output
        top_p=0.9,
        stop=["<|end|>", "</s>"],    # prevent model from rambling
    )


    # print(response)
    raw_output = response["choices"][0]["text"]

    print("\n==== RAW MODEL OUTPUT ====")
    print(raw_output)
    print("==== END RAW OUTPUT ====\n")

    return extract_json(raw_output)

# Extract raw text from PDF using PyMuPDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text("text") for page in doc)
    return text if text.strip() else "Error: No extractable text found in PDF."

# Load LLaMA 3 GGUF Model

# Path to your GGUF model
model_path = "/content/models/phi3.5/Phi-3.5-mini-instruct.Q8_0.gguf"
# Load model
model = load_phi_model(model_path)

if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/19.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))


llama_init_from_model: n_ctx_per_seq (4096) < n_ctx_train (131072) -- the full capacity of the model will not be utilized



==== RAW MODEL OUTPUT ====

JSON:
```json
{
  "Title": "Predicting movies’ eudaimonic and hedonic scores: A machine learning approach using metadata, audio and visual features",
  "Authors": [
    "Elham Motamedi",
    "Danial Khosh Kholgh",
    "Sorush Saghari",
    "Mehdi Elahi",
    "Francesco Barile",
    "Marko Tkalcic"
  ],
  "DOI": "10.1016/j.infpro.2023.103610",
  "Keywords": [
    "Eudaimonia",
    "Hedonia",
    "Machine learning approach",
    "Movie recommender systems"
  ],
  "Abstract": "In the task of modeling user preferences for movie recommender systems, recent research has demonstrated the benefits of describing movies with their eudaimonic and hedonic scores (E and H scores), which reflect the depth of their message and the level of fun experience they provide, respectively. So far, the labeling of movies with their E and H scores has been done manually using a dedicated instrument (a questionnaire), which is time-consuming. To address this issue, we propose an aut

In [ ]:
if model is None:
    print("Model loading failed. Exiting.")
    exit()  # Or handle the error appropriately
else:
# Extract text from PDF
  extracted_text = extract_text_from_pdf("/content/20.pdf")

  if extracted_text.startswith("Error:"):
      print("No extractable text found in the PDF.")
  else:
    # Extract metadata using the LLaMA 3 model
      metadata = extract_metadata(model, extracted_text)
      print("\n==== Extracted Metadata ====")
      print(json.dumps(metadata, indent=4))



==== RAW MODEL OUTPUT ====
 {
  "Title": "Deep Learning-based Extraction of Algorithmic Metadata in Full-Text Scholarly Documents",
  "Authors": ["Iqra Safdera", "Saeed-Ul Hassana", "Anna Visvizib", "Thanapon Norasetc", "Raheel Nawazd", "Suppawong Tuarobc"],
  "DOI": "Not provided",
  "Keywords": ["Knowledge-based Systems", "Algorithmic Metadata", "Algorithm Search", "Deep Learning", "Bi-Directional LSTM", "Information Retrieval", "Full-text Articles"],
  "Abstract": "The advancements of search engines for traditional text documents have enabled the effective retrieval of massive textual information in a resource-efficient manner. However, such conventional search methodologies often suffer from poor retrieval accuracy especially when documents exhibit unique properties that behoove specialized and deeper semantic extraction. Recently, AlgorithmSeer, a search engine for algorithms has been proposed, that extracts pseudo-codes and shallow textual metadata from scientific publications a